# Notes:

You are banned from using loops (`for` or `while` or any other) for this entire workshop!

You shouldn't be using loops almost ever with pandas in any case, so break out of the habit now.

## 1. DataFrame basics


Consider the following Python dictionary `data` and Python list `labels`:

``` python
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
```
(This is just some meaningless data I made up with the theme of animals and trips to a vet.)

**1.** Create a DataFrame `df` from this dictionary `data` which has the index `labels`.

**2.** Select only the rows where visits are 3 or more. Which types of animals are these?

**3.** Select the rows where visists are 3 and the animal is a cat

**4.** Calculate the sum of all visits in `df` (i.e. the total number of visits).

**5.** Calculate the mean age for each different animal in `df`.

**6.** Append a new row 'k' to `df` with your choice of values for each column. Then delete that row to return the original DataFrame.



In [2]:
import pandas as pd
import numpy as np

In [3]:
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [4]:
df = pd.DataFrame(data,index=labels)
df#.index = labels

,animal,age,visits,priority
a,cat,2.5,1,yes
b,cat,3.0,3,yes
c,snake,0.5,2,no
d,dog,NaN,3,yes
e,dog,5.0,2,no
f,cat,2.0,3,no
g,snake,4.5,1,no
h,cat,NaN,1,yes
i,dog,7.0,2,no
j,dog,3.0,1,no


In [5]:
selection = df[df.visits >= 3]
selection.animal.unique()


array(['cat', 'dog'], dtype=object)

In [6]:
df.loc[(df.visits==3) & (df.animal == 'cat')]


,animal,age,visits,priority
b,cat,3.0,3,yes
f,cat,2.0,3,no


In [7]:
df.visits.sum()

19

In [8]:
df.groupby('animal')['age'].mean()

animal
cat      2.5
dog      5.0
snake    2.5
Name: age, dtype: float64

In [9]:
df.loc['k']=['cat',8.2,10,'no']
df

,animal,age,visits,priority
a,cat,2.5,1,yes
b,cat,3.0,3,yes
c,snake,0.5,2,no
d,dog,NaN,3,yes
e,dog,5.0,2,no
f,cat,2.0,3,no
g,snake,4.5,1,no
h,cat,NaN,1,yes
i,dog,7.0,2,no
j,dog,3.0,1,no


In [10]:
df.drop(['k'])

,animal,age,visits,priority
a,cat,2.5,1,yes
b,cat,3.0,3,yes
c,snake,0.5,2,no
d,dog,NaN,3,yes
e,dog,5.0,2,no
f,cat,2.0,3,no
g,snake,4.5,1,no
h,cat,NaN,1,yes
i,dog,7.0,2,no
j,dog,3.0,1,no


# 2.1 Shifty problem

You have a DataFrame `df` with a column 'A' of integers. For example:
```python
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
```

How do you filter out rows which contain the same integer as the row immediately above?

You should be left with a column containing the following values:

```python
1, 2, 3, 4, 5, 6, 7
```

### Hint: use the `shift()` method

In [35]:
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})

In [37]:
df['shifted_A']=df.shift(periods=1,axis=0)
df

df = df.loc[(df.shifted_A != df.A)]
df = df.drop(columns=['shifted_A'])
#df = df.reset_index(drop=True)
df

,A
0,1
1,2
2,3
3,4
4,5
5,6
6,7


# 2.2 columns sum min

Suppose you have DataFrame with 10 columns of real numbers, for example:

```python
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))
```
Which column of numbers has the smallest sum? Return that column's label.

In [38]:
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))

In [58]:
total = df.sum(axis=0)
total.loc[(total == total.min())]

e    1.269815
dtype: float64

In [59]:
df.sum().idxmin()

'e'

# 2.3 Duplicates

How do you count how many unique rows a DataFrame has (i.e. ignore all rows that are duplicates)?

**hint:** There's a method for to find duplicate rows for you

In [75]:
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})


In [76]:
dff = df.duplicated(subset=None, keep='first') == False
dff.sum()

7

In [79]:
df.drop_duplicates().count()

A    7
dtype: int64

# 2.4 Group Values

A DataFrame has a column of groups 'grps' and and column of integer values 'vals': 

```python
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
```
For each *group*, find the sum of the three greatest values.  You should end up with the answer as follows:
```
grps
a    409
b    156
c    345
```

In [80]:
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})

In [90]:
df.groupby('grps')['vals'].apply(lambda grp: grp.nlargest(2).sum())

grps
a    397
b    111
c    322
Name: vals, dtype: int64

# 3. Cleaning Data

### Making a DataFrame easier to work with

It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

```python
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})
```

Formatted, it looks like this:

```
            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis           NaN          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"
```

**1.** Some values in the the **FlightNumber** column are missing (they are `NaN`). These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Modify `df` to fill in these missing numbers and make the column an integer column (instead of a float column).

In [14]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})

In [17]:
df['shifted_flight'] = df['FlightNumber'].shift(periods=1,axis=0)
#df['FlightNumber'] = df['FlightNumber'].replace(np.nan, (df['shifted_flight']) + 10.0)
#df.FlightNumber = df.FlightNumber.replace(np.nan, (df.shifted_flight + 10))
#df = df.drop(columns=['shifted_flight'])
#df

In [18]:
df.FlightNumber[df.FlightNumber.isna()] = df.shifted_flight + 10
df = df.drop(columns=['shifted_flight'])#df.drop('shifted_flight',axis=1)
df.FlightNumber =df.FlightNumber.astype(int)


In [19]:
df

,From_To,FlightNumber,RecentDelays,Airline,shifted_flight
0,LoNDon_paris,10045,"[23, 47]",KLM(!),NaN
1,MAdrid_miLAN,10055,[],<Air France> (12),10045.0
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),NaN
3,Budapest_PaRis,10075,[13],12. Air France,10065.0
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",NaN


# 3.2 column splitting

The **From\_To** column would be better as two separate columns! Split each string on the underscore delimiter `_` to make two new columns `From` and `To` to your dataframe.

In [23]:
df[['From','to']] = df.From_To.str.split('_',1,expand=True)

In [24]:
df


,From_To,FlightNumber,RecentDelays,Airline,shifted_flight,From,to
0,LoNDon_paris,10045,"[23, 47]",KLM(!),NaN,LoNDon,paris
1,MAdrid_miLAN,10055,[],<Air France> (12),10045.0,MAdrid,miLAN
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),NaN,londON,StockhOlm
3,Budapest_PaRis,10075,[13],12. Air France,10065.0,Budapest,PaRis
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",NaN,Brussels,londOn


# 3.3 Clean Text

Make the text in your dataframe:

- From and To columns should be lowercase with only first letter capitalized

- In the **Airline** column, you can see some extra puctuation and symbols have appeared around the airline names. Pull out just the airline name. E.g. `'(British Airways. )'` should become `'British Airways'`.

In [54]:
df.Airline = df.Airline.str.replace('[#,@,&,",>,<,1,2,3,4,5,6,7,8,9,.,?,!]', '') 

In [55]:
df

,From_To,FlightNumber,RecentDelays,Airline,shifted_flight,From,to
0,LoNDon_paris,10045,"[23, 47]",KLM,NaN,LoNDon,paris
1,MAdrid_miLAN,10055,[],Air France,10045.0,MAdrid,miLAN
2,londON_StockhOlm,10065,"[24, 43, 87]",British Airways,NaN,londON,StockhOlm
3,Budapest_PaRis,10075,[13],Air France,10065.0,Budapest,PaRis
4,Brussels_londOn,10085,"[67, 32]",Swiss Air,NaN,Brussels,londOn


# Exercise 4.1: Column Splitting

Given the unemployment data in `data/country_total.csv`, split the `month` column into two new columns: a `year` column and a `month` column, both integers

In [96]:
import pandas as pd
df_ct = pd.DataFrame(pd.read_csv("data/country_total.csv"))
df_ct

,country,seasonality,month,unemployment,unemployment_rate
0,at,nsa,1993.01,171000,4.5
1,at,nsa,1993.02,175000,4.6
2,at,nsa,1993.03,166000,4.4
3,at,nsa,1993.04,157000,4.1
4,at,nsa,1993.05,147000,3.9
...,...,...,...,...,...
20791,uk,trend,2010.06,2429000,7.7
20792,uk,trend,2010.07,2422000,7.7
20793,uk,trend,2010.08,2429000,7.7
20794,uk,trend,2010.09,2447000,7.8


In [111]:
df_ct = df_ct.rename(columns={'month' : 'year_month'})
df_ct.dtypes

country               object
seasonality           object
year_month           float64
unemployment           int64
unemployment_rate    float64
dtype: object

In [127]:
df_ct.year_month = df_ct.year_month.astype(str)
df_ct[['year','month']] = df_ct.year_month.str.split('.',1,expand=True)

In [128]:
df_ct 

,country,seasonality,year_month,unemployment,unemployment_rate,year,month
0,at,nsa,1993.01,171000,4.5,1993,01
1,at,nsa,1993.02,175000,4.6,1993,02
2,at,nsa,1993.03,166000,4.4,1993,03
3,at,nsa,1993.04,157000,4.1,1993,04
4,at,nsa,1993.05,147000,3.9,1993,05
...,...,...,...,...,...,...,...
20791,uk,trend,2010.06,2429000,7.7,2010,06
20792,uk,trend,2010.07,2422000,7.7,2010,07
20793,uk,trend,2010.08,2429000,7.7,2010,08
20794,uk,trend,2010.09,2447000,7.8,2010,09


# 4.2 Group Statistics

Given the unemployment data in `data/country_sex_age.csv`, give the average unemployment rate for:

- Each gender
- Each Age Group
- Both Together

**HINT:** The `seasonality` column makes it such that the data is repeated for each method of calculating unemployment (`nsa`, `trend`, etc.). Can you ignore this and group over it? Or should you take the average for each?

In [134]:
import pandas as pd
df_csa = pd.DataFrame(pd.read_csv("data/country_sex_age.csv"))
df_csa

,country,seasonality,sex,age_group,month,unemployment,unemployment_rate
0,at,nsa,f,y25-74,1993.01,61000,4.5
1,at,nsa,f,y25-74,1993.02,62000,4.5
2,at,nsa,f,y25-74,1993.03,62000,4.5
3,at,nsa,f,y25-74,1993.04,63000,4.6
4,at,nsa,f,y25-74,1993.05,63000,4.6
...,...,...,...,...,...,...,...
83155,uk,trend,m,y_lt25,2010.06,518000,21.1
83156,uk,trend,m,y_lt25,2010.07,513000,20.8
83157,uk,trend,m,y_lt25,2010.08,509000,20.5
83158,uk,trend,m,y_lt25,2010.09,513000,20.7


In [143]:

df_csa.groupby(['sex','seasonality']).agg({'unemployment_rate' : ['mean']})

unemployment_rate
                             mean
sex seasonality                  
f   nsa                 13.179216
    sa                  12.908743
    trend               12.862163
m   nsa                 11.708432
    sa                  11.662290
    trend               11.643015

In [144]:
df_csa.groupby(['age_group','seasonality']).agg({'unemployment_rate' : ['mean']})

unemployment_rate
                                   mean
age_group seasonality                  
y25-74    nsa                  6.870746
          sa                   6.918446
          trend                6.926319
y_lt25    nsa                 18.016902
          sa                  17.690707
          trend               17.617224

In [142]:
df_csa.groupby(['sex','age_group','seasonality']).agg({'unemployment_rate' : ['mean']})



unemployment_rate
                                       mean
sex age_group seasonality                  
f   y25-74    nsa                  7.539839
              sa                   7.579982
              trend                7.579934
    y_lt25    nsa                 18.818593
              sa                  18.323837
              trend               18.231025
m   y25-74    nsa                  6.201653
              sa                   6.256909
              trend                6.272703
    y_lt25    nsa                 17.215211
              sa                  17.067671
              trend               17.013327

In [ ]:
# we have to use thej seasonality because the data have change when we take it off

# 4.3 Estimating group size

Given that we have the unemployment **rate** as a % of total population, and the number of total unemployed, we can estimate the total population.

Give an estimate of the total population for men and women in each age group.

Does this change depending on the unemployment seasonality calculation method?

In [145]:
df_csa['population'] = df_csa.unemployment / (df_csa.unemployment_rate / 100)

In [148]:
df_csa

,country,seasonality,sex,age_group,month,unemployment,unemployment_rate,population
0,at,nsa,f,y25-74,1993.01,61000,4.5,1.355556e+06
1,at,nsa,f,y25-74,1993.02,62000,4.5,1.377778e+06
2,at,nsa,f,y25-74,1993.03,62000,4.5,1.377778e+06
3,at,nsa,f,y25-74,1993.04,63000,4.6,1.369565e+06
4,at,nsa,f,y25-74,1993.05,63000,4.6,1.369565e+06
...,...,...,...,...,...,...,...,...
83155,uk,trend,m,y_lt25,2010.06,518000,21.1,2.454976e+06
83156,uk,trend,m,y_lt25,2010.07,513000,20.8,2.466346e+06
83157,uk,trend,m,y_lt25,2010.08,509000,20.5,2.482927e+06
83158,uk,trend,m,y_lt25,2010.09,513000,20.7,2.478261e+06


In [151]:
df_csa.groupby(['sex','age_group','seasonality']).agg({'population' : ['mean']})

population
                                   mean
sex age_group seasonality              
f   y25-74    nsa          3.006779e+06
              sa           3.360649e+06
              trend        3.289520e+06
    y_lt25    nsa          5.270112e+05
              sa           5.876348e+05
              trend        5.848969e+05
m   y25-74    nsa          4.110194e+06
              sa           4.533201e+06
              trend        4.423824e+06
    y_lt25    nsa          6.354751e+05
              sa           6.867673e+05
              trend        6.809973e+05

# 5.1 Tennis

In `data/tennis.csv` you have games that Roger Federer played against various opponents. Questions:

1. How many games did Federer win?

2. What is Federer's win/loss ratio?

3. Who were Federer's top 5 opponents?

In [250]:
tennis = pd.read_csv('data/tennis.csv')

In [251]:
tennis.win.sum()

972

In [252]:
tennis.win.count()

1179

In [253]:
ratio =  tennis.win.sum() / (tennis.win.count() - tennis.win.sum())
ratio

4.695652173913044

In [254]:
tennis[tennis.win==False].groupby(['winner']).winner.count().nlargest(5)


winner
Rafael Nadal        18
Novak Djokovic      13
Andy Murray         10
David Nalbandian     8
Lleyton Hewitt       8
Name: winner, dtype: int64

# 5.2 Over time

1. What was Federer's best year? In terms of money, and then in terms of number of wins

2. Did Federer get better or worse over time?

In [255]:
tennis = tennis.rename(columns={'tournament prize money' : 'tournament_prize_money'})

In [256]:
tennis.tournament_prize_money = tennis.tournament_prize_money.replace('[A,$,,]','',regex=True).replace('',0).astype(float)


385788068.0

In [265]:
tennis[(tennis.win==True) & (tennis['tournament round']=='F')].groupby(['year']).tournament_prize_money.sum().sort_values(ascending=False)

year
2007    7245735.0
2006    7221635.0
2004    6229377.0
2005    4733250.0
2010    4561045.0
2012    3971120.0
2003    3026502.0
2009    2938500.0
2011    2579000.0
2008    1819800.0
2002     540600.0
2001      54000.0
1999      14400.0
Name: tournament_prize_money, dtype: float64

In [266]:
tennis[(tennis.win==True) & (tennis['tournament round']=='F')].groupby(['year']).tournament.count().sort_values(ascending=False)

year
2006    12
2005    11
2004    11
2007     8
2003     7
2012     6
2010     5
2011     4
2009     4
2008     4
2002     3
2001     1
1999     1
Name: tournament, dtype: int64

# 5.3 Total money won

In the data, you'll find the `tournament round`, one value of which, `F` indicates the final.

Assuming Federer wins the money in the `tournament prize money` if he wins a final in a tournament, how much money has Federer made in tournaments in this dataset?

In [267]:
tennis[(tennis.win==True) & (tennis['tournament round']=='F')].tournament_prize_money.sum()

44934964.0